In [177]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [214]:
DROWSY_THRESHOLD = 15

In [215]:
face_classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
eyes_classifier = cv2.CascadeClassifier('./haarcascade_eye.xml')

model=load_model('../model/henry_model.h5')
model.summary()

def to_grayscale_then_rgb(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.grayscale_to_rgb(image)
    return image

close_frame = 0
prev_eyes = []
def face_ROI(frame):
    
    global close_frame, prev_eyes
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(frame_gray, 1.3, 3, minSize=(30, 30))
    
    for (x,y,w,h) in faces:
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        face_gray = frame_gray[y:y+h, x:x+w]
        face = frame[y:y+h, x:x+w]
#       cv2.imwrite("face.jpg", face)
        eyes = eyes_classifier.detectMultiScale(face_gray)
    
        new_eyes = []
        for (x2,y2,w2,h2) in eyes:
            
            if y2+h2/2 > h/2: # skip if the eye is at the bottom
                continue
            
            new_eyes.append((x2,y2,w2,h2))
            
            

        if new_eyes:
            prev_eyes = new_eyes
        else:
            close_frame += 1
            
        for (x2,y2,w2,h2) in prev_eyes:
            cv2.rectangle(face,(x2,y2),(x2+w2,y2+h2),(0,0,255),2)
            eye = face_gray[y2:y2+h2, x2:x2+w2]
            eye = cv2.cvtColor(eye, cv2.COLOR_GRAY2BGR)
            try:
                eye = cv2.resize(eye, (50, 50))
            except Exception as e:
                break
            eye = eye/255
            eye = np.array(eye)
            eye = np.expand_dims(eye, axis=0)
            prediction = model.predict(eye)
#             print(prediction[0][0])
            if(prediction[0][0] < 0.5):
                close_frame += 1
            else:
                close_frame = max(0, close_frame-1)
            print(close_frame)
            
        if close_frame >= DROWSY_THRESHOLD:
            cv2.putText(frame, 
                        "DROWSY!!!", 
                        (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, 
                        (0, 255, 255), 
                        2, 
                        cv2.LINE_4)    

    cv2.imshow('TEAM12', frame)

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 16)        448       
_________________________________________________________________
max_pooling2d_105 (MaxPoolin (None, 24, 24, 16)        0         
_________________________________________________________________
batch_normalization_93 (Batc (None, 24, 24, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 32)        4640      
_________________________________________________________________
max_pooling2d_106 (MaxPoolin (None, 11, 11, 32)        0         
_________________________________________________________________
batch_normalization_94 (Batc (None, 11, 11, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 64)        

In [216]:
vid = cv2.VideoCapture(0)
print(vid.get(cv2.CAP_PROP_FPS))
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame,(500,250))
    face_ROI(frame)
    # Display the resulting frame
#     cv2.imshow('TEAM12', frame)
      
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    # After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

5.0
0
0
0
0
0
0
0
0
0
0
0
0
2
3
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
4
6
8
10
12
14
16
16
16
16
16
18
18
20
20
20
22
24
24
26
28
30
32
32
32
34
36
36
36
38
40
42
44
44
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [31]:
img = cv2.imread("eye.jpg")
img = cv2.resize(img, (50, 50))
img = np.array(img)
img = np.expand_dims(img, axis=0)
# destroy all windows
prediction = model.predict(img)
print(prediction)

[[0.]]
